# Исследование надёжности заёмщиков

По заказу кредитного отдела банка необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

**План проекта**
    
    1. Исходные данные, общая информация
    2. Предобработка данных
    3. Влияние отдельных признаков на возврат кредита в срок
    Заключение

### 1. Исходные данные, общая информация

In [1]:
!pip install pymystem3
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import collections
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe().round(2)

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,19351.00,21525.00,21525.00,21525.00,21525.00,19351.00
mean,0.54,63046.50,43.29,0.82,0.97,0.08,167422.30
std,1.38,140827.31,12.57,0.55,1.42,0.27,102971.57
min,-1.00,-18388.95,0.00,0.00,0.00,0.00,20667.26
25%,0.00,-2747.42,33.00,1.00,0.00,0.00,103053.15
50%,0.00,-1203.37,42.00,1.00,0.00,0.00,145017.94
75%,1.00,-291.10,53.00,1.00,1.00,0.00,203435.07
max,20.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


In [5]:
data.days_employed.isna().sum() 

2174

In [6]:
data.total_income.isna().sum()

2174

In [7]:
strange_gr = data[data.total_income.isna() & data.days_employed.isna()] # пропуски по трудовому стажу и доходу относятся 
                                                                        # к одной и той же группе клиентов
strange_gr.head()    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [8]:
print('Процент проблемных клиентов (имели задолженность по кредиту) по всему массиву', 
                                                        '{0:.1%}'.format(data.debt.value_counts()[1]/data.shape[0]))
print('Процент проблемных клиентов (имели задолженность по кредиту) в группе с пропусками', 
                                                        '{0:.1%}'.format(strange_gr.debt.value_counts()[1]/strange_gr.shape[0]))

Процент проблемных клиентов (имели задолженность по кредиту) по всему массиву 8.1%
Процент проблемных клиентов (имели задолженность по кредиту) в группе с пропусками 7.8%


In [9]:
data.children.value_counts()  # 76 клиентов имеют 20 детей 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [10]:
strange_days_employed = data[abs(data.days_employed) > 365*60] # массив со слишком большими значениями трудового стажа
strange_days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [11]:
strange_days_employed.income_type.value_counts() # с данными пенсионеров что-то не так, если рассмотреть пенсионеров в этой
                                                 # группе по возрасту, тоже возникают вопросы, хотя есть пенсии по инвалидности,
                                                 # пенсии по потере кормильца. Стоило бы добавить данные по типу пенсии

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [12]:
var_type = ['количественная, целое',
            'количественная, вещественное',
            'количественная, целое',
            'категориальная',
            'категориальная',
            'категориальная',  
            'категориальная',
            'категориальная',
            'категориальная',
            'логическая',
            'количественная, вещественное', 
            'категориальная']

descr = ['кол-во детей', 'трудовой стаж', 'возраст', 'образование', 'код образования', 'семейный статус', 
         'код семейного статуса', 'пол', 'тип занятости', 'задолженность', 'доход', 'цель кредита']

d = {'col':list(data.columns), 'descrip':descr, 'type':var_type, 
               'uniq_quant':[len(data[x].value_counts()) for x in list(data.columns)]}
unique_quant_in_col = pd.DataFrame(d)
unique_quant_in_col

,col,descrip,type,uniq_quant
0,children,кол-во детей,"количественная, целое",8
1,days_employed,трудовой стаж,"количественная, вещественное",19351
2,dob_years,возраст,"количественная, целое",58
3,education,образование,категориальная,15
4,education_id,код образования,категориальная,5
5,family_status,семейный статус,категориальная,5
6,family_status_id,код семейного статуса,категориальная,5
7,gender,пол,категориальная,3
8,income_type,тип занятости,категориальная,8
9,debt,задолженность,логическая,2


**Вывод**

<span style="font-family: Calibri; font-size:1.25em;color:navy;"> 1. Файл 'data.csv' содержит двумерный массив с данными по клиентам банка в количестве 21525 человек без учета возможных дубликатов по 12 признакам. Из 12 признаков 4 соответствуют переменным количественного типа, 8 - категориального или логического типа. В таблице выше приведен тип переменной для каждого признака, а также количество уникальных значений.   
     
<span style="font-family: Calibri; font-size:1.25em;color:navy;"> 2. Для порядка 10% от обшего числа в представленном файле -  2174 клиентов (здесь и далее допускается возможность дубликатов) одновременно отсутствуют данные по трудовому стажу и доходу. Данная группа разнородна, представлена клиентами разных возрастов и типов занятости, не может быть описана как 'молодые, безработные люди'. Процент проблемных клиентов в этой группе практически такой же, как и во всем массиве. Вероятно указанные пропуски вызваны человеческим фактором при формировании массива, техническим сбоем.   
    
<span style="font-family: Calibri; font-size:1.25em;color:navy;"> 3. Вопросы по значениям отдельных признаков:   
        
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - количество детей

Переменная принимает значения от -1 до 20. -1 крайне вероятно ошибка ввода, заменим ниже на 1. 
По [данным](https://russian.rt.com/russia/article/372844-chislo-mnogodetnyh-semei-rossiya) СМИ RT со ссылкой на официальных представителей власти в России на начало 2017 года было не более 1000 семей, воспитывающих 11 детей и более. Можно было бы предположить, что банк предоставляет особые условия таким семьям, но то, что у всех клиентов (у одного банка их 76) из этой особой немногочисленной группы именно по 20 детей - вероятность стремится к 0. Примем данное значение за ошибку ввода, ниже заменим 20 на 2.        
        
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - трудовой стаж

Помимо пропусков, данная переменная принимает отрицательные значения и значения вне возможного диапозона - максимальное значение - более 400 тыс. дней, то есть более 1000 лет. Примем за ошибку ввода отрицательные значения, заменим их на абсолютные. 
Группа клиентов со слишком большими значениями трудового стажа по типу занятости - пенсионеры. Для нее запишем трудовой стаж, как возраст - 16. Официально на текущий момент оформление по трудовому договору за редким исключением возможно с 16 лет. Здесь вероятно искажение в сторону завышения трудового стажа, но учитывая тип занятости в данной группе, это несущественно, хотя к возрасту пенсионеров тоже есть вопросы.
Значения данного признака наиболее противоречивы. Вероятно это ошибка ввода данных, обработки данных (возможно неверная формула), технический сбой.
        
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - возраст

У 101 клиента исходного массива возраст - 0. Позже скорректируем эти данные. 
        
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - образование

Присутствует разное написание одинаковых значений, кол-во уникальных значений по этому признаку и по коду образования различно. Позже скорректируем эти данные.
        
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - пол

Одно единственное значение добавляет третий пол. Оставим его, как есть.

<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - тип занятости

Значение признака 'компаньон' просит расшифровки. 
    
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - доход

Описательные статистики непропущенных значений не вызывают сомнений. Max/min~108, среднее 167тр, медиана 145тр. Обработаем пропуски ниже.
                
<span style="font-family: Calibri; font-size:1.25em;color:navy;">  - цель кредита

В массиве 38 вариаций значений данного признака, список стоит сократить.


## 2. Предобработка данных

### Обработка пропусков

In [13]:
data.children = abs(data.children).replace(20, 2) # корректируем признак количество детей: -1 -> 1,   20 -> 2  

In [14]:
# заменим пропуски признака доход на медиану дохода по типу занятости из имеющихся данных
# выбор медианы - медиана меньше, чем среднее. Даже если занижение - это безопаснее для скоринга

j = list(strange_gr.income_type.value_counts().index) # список типов занятых в пропусках

median_inc = data.groupby(data.income_type).total_income.median() # серия медиан дохода с индексом по типу занятости

for x in j:
    data.loc[(data.income_type==x) & (data.total_income.isnull()), 'total_income'] = median_inc[x] # замена пропусков на медиану
                                                        # дохода по типу занятости доступных данных исходного массива        

In [15]:
# заменим 101 нулевое значение признака возраст на медиану возраста по типу занятости из имеющихся данных
# выбор медианы - медиана и среднее почти совпадают

jobs = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий'] # список типов занятых для возраста 0

median_age = data.groupby(data.income_type).dob_years.median() # серия медиан возраста с индексом по типу занятости

for x in jobs:
    data.loc[(data.income_type==x) & (data.dob_years==0), 'dob_years'] = median_age[x]


In [16]:
# заменим значения трудового стажа на положительные, заменим значения трудового стажа, которые не коррелируют с возрастом
data.days_employed = abs(data.days_employed)
data.loc[abs(data.days_employed) > 365*60, 'days_employed'] = (data.dob_years - 16)*365

j = list(strange_gr.income_type.value_counts().index) # список типов занятых в пропусках

median_days_employed = data.groupby(data.income_type).days_employed.median() # серия медиан трудового стажа с индексом по типу
                                                                             # занятости

for x in j:
    data.loc[(data.income_type==x) & (data.days_employed.isnull()), 'days_employed'] = median_days_employed[x] 
                      # замена пропусков на медиану трудового стажа по типу занятости доступных данных исходного массива        
                                                                                  


data.describe().round()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.0,21525.0,21525.0,21525.0,21525.0,21525.0,21525.0
mean,0.0,4719.0,43.0,1.0,1.0,0.0,165225.0
std,1.0,5655.0,12.0,1.0,1.0,0.0,98044.0
min,0.0,24.0,19.0,0.0,0.0,0.0,20667.0
25%,0.0,1026.0,34.0,1.0,0.0,0.0,107798.0
50%,0.0,1994.0,43.0,1.0,0.0,0.0,142594.0
75%,1.0,5324.0,53.0,1.0,1.0,0.0,195550.0
max,5.0,21170.0,75.0,4.0,4.0,1.0,2265604.0


Пропуски заполнены значениями медиан по типу занятости.


### Замена типа данных

In [17]:
# Трудовой стаж удобнее воспринимать в годах, переименуем колонку и разделим значения на 365, изменим тип float на int.
data = data.rename(columns={'days_employed': 'years_employed'})
data.years_employed = data.years_employed/365
data.years_employed = data.years_employed.astype('int')

# Возраст в процессе операций сменил тип на float, восстановим тип int.
data.dob_years = data.dob_years.astype('int')

# Доход удобнее воспринимать как целое число, изменим также тип на int.
data.total_income = data.total_income.astype('int')

# Приведем к нижнему регистру все описательные категориальные переменные.
data.education = data.education.str.lower()
data.income_type = data.income_type.str.lower()
data.family_status = data.family_status.str.lower()
data.purpose = data.purpose.str.lower()

data.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,37,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Изменили тип количественных признаков трудового стажа, возраста и дохода c вещественного на целый с применением метода astype, так как пропуски уже обработаны и преобразовать нужно число. Также привели к нижнему регистру категориальные переменные. Значения трудового стажа вывели в годах. Преобразованный массив читается легче.

### Обработка дубликатов

In [18]:
# Ранее значения всех категориальных описательных переменных были приведены к нижнему регистру, что позволило выделить 
# корректное количество уникальных значений по признаку образование.
# Для поиска дубликатов воспользуемся методом duplicated().

print('Количество дубликатов в массиве:', data.duplicated().sum())
print('Количество полностью идентичных строк:', data.duplicated(keep = False).sum())
print('Доля дубликатов: {:.1%}'.format(data.duplicated().sum() / len(data)))

Количество дубликатов в массиве: 71
Количество полностью идентичных строк: 137
Доля дубликатов: 0.3%


In [19]:
data[data.duplicated(keep = False)].sort_values('dob_years', ascending = False).head() # Посмотрим, как выглядят дубликаты.
# Скорее всего подобные строки появились в результате повторного ввода. Массив можно было бы защитить от появления 
# идентичных строк на этапе формирования.

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7938,0,44,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
9604,0,44,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
6537,0,44,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
5865,0,44,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9528,0,44,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью


In [20]:
data = data.drop_duplicates().reset_index(drop = True)
print('Количество дубликатов в массиве', data.duplicated().sum())

Количество дубликатов в массиве 0


После преобразования массив не содержит полностью идентичных строк. Между тем наличие дубликатов характеризует качество исходного массива, дает дополнительное основание сомневаться в корректности данных.

### Лемматизация

In [21]:
m = Mystem()
lemmas = []
for purp in data.purpose:
    for word in purp.split():                    # Получаем список лемм длиной более двух букв 
        lem = m.lemmatize(word)                  # каждого слова из формулировки цели.
        if len(lem[0]) > 2:
            lemmas.append(lem[0])

word_frequency = collections.Counter(lemmas)
word_frequency

Installing mystem to /Users/julia/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


Counter({'покупка': 5897,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 964,
         'свой': 2230,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

In [22]:
# Сопоставим частотность лемм и формулировок цели. 
data.purpose.value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [23]:
# Формулировки цели кредита очевидно 'живые' на усмотрение клиента, а их выбор стоит ограничить списком.
# Наиболее вариативной категорией является недвижимость - она может быть жилой(жильем), коммерческой, с ней могут быть операции, 
# кто-то планирует строительство, а кто-то в принципе не уточняет. Фиксированный список целей мог бы исключить трактовки.
# Между тем сформируем список категорий целей кредита из имеющихся данных. На данном этапе необходимо выделить категории
# жилье и недвижимость, позже объединим их.
purpose_category = ['недвижимость', 'автомобиль', 'свадьба', 'образование', 'ремонт', 'жилье']

def lemma(phrase):
    for word in phrase.split():
        lem = m.lemmatize(word)
        if lem[0] in purpose_category:
            return lem[0]
    
data['purpose_category'] = data.purpose.apply(lemma)
data


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,37,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,12,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье
21450,0,51,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21451,1,5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21452,3,8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


In [24]:
data.purpose_category.value_counts()

недвижимость    6351
автомобиль      4306
образование     4013
жилье           3853
свадьба         2324
ремонт           607
Name: purpose_category, dtype: int64

In [25]:
# Получившиеся 6 категорий целей кредита сократим до 5. Категории жилье и недвижимость объединим в одну - недвижимость. 
data.loc[data['purpose_category'] == 'жилье', 'purpose_category'] = 'недвижимость'
data.purpose_category.value_counts()

недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_category, dtype: int64

В массив был добавлен столбец с категорией цель кредита, всего выделено 5 категорий на основе формулировки цели кредита. Категория недвижимость могла бы быть более детализированной.

### Категоризация данных

In [26]:
data.describe()

,children,years_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,12.512445,43.474690,0.817097,0.973898,0.081150,1.653196e+05
std,0.756069,15.646559,12.226883,0.548674,1.421567,0.273072,9.818730e+04
min,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,2.000000,33.000000,1.000000,0.000000,0.000000,1.076230e+05
50%,0.000000,5.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,14.000000,53.000000,1.000000,1.000000,0.000000,1.958202e+05
max,5.000000,58.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [27]:
# Отнесем всех клиентов к нескольким возрастным группам, группам по доходу, наличию детей.
# Категоризация по возрасту и доходу улучшает читаемость массива. Также выделение групп по доходу  
# понадобится для выявления зависимости уровня дохода и риска невозврата кредита в срок. 

def age_cat(x):
    if x <= 33:
        return 'до 33'
    if 34 <= x <= 42:
        return '34-42'
    if 43 <= x <= 53:
        return '43-53'
    if x >= 54 :
        return '54+'
   
data['age_group'] = data.dob_years.apply(age_cat)


def income_cat(x):
    if x <= 107600:
        return '(1) до 107,6'
    elif 107601 <= x <= 142600:
        return '(2) 107,6-142,6'
    elif 142601 <= x <= 195800:
        return '(3) 142,6-195,8'
    else:
        return '(4) 195,8+'

data['income_group'] = data.total_income.apply(income_cat)

def def_cildren_cat(x):
    if x != 0:
        return 'есть дети'
    else:
        return 'нет детей'
    
data['children_cat'] = data.children.apply(def_cildren_cat)

def fam_stat_cat(x):
    if x == 'не женат / не замужем':
        return 'не было опыта семейной жизни'
    else:
        return 'был опыт семейной жизни'

data['family_cat'] = data.family_status.apply(fam_stat_cat)

print('Распределение клиентов по возрастным группам')
print(data.age_group.value_counts())
print()
print('Распределение клиентов по группам дохода')
print(data.income_group.value_counts())
print()
print('Распределение клиентов по наличию детей')
print(data.children_cat.value_counts())
print()
print('Распределение клиентов по наличию опыта семейной жизни')
print(data.family_cat.value_counts())

Распределение клиентов по возрастным группам
43-53    5448
до 33    5366
34-42    5365
54+      5275
Name: age_group, dtype: int64

Распределение клиентов по группам дохода
(2) 107,6-142,6    5481
(4) 195,8+         5365
(1) до 107,6       5363
(3) 142,6-195,8    5245
Name: income_group, dtype: int64

Распределение клиентов по наличию детей
нет детей    14091
есть дети     7363
Name: children_cat, dtype: int64

Распределение клиентов по наличию опыта семейной жизни
был опыт семейной жизни         18644
не было опыта семейной жизни     2810
Name: family_cat, dtype: int64


Были выделены категории данных по возрасту, доходу, наличию детей с границами по квантилям. Удивительно, что 66% клиентов не имееют детей, при том, что 75% старше 33-х лет.

## 3. Влияние отдельных признаков на возврат кредита в срок

**- Наличие детей**

In [28]:
pr1 = data.loc[(data.children_cat == 'есть дети')&(data.debt == 1), 'debt'].sum()/data[data.children_cat == 'есть дети'].shape[0]
print('% проблемных клиентов среди тех, у кого есть дети', '{:.1%}'.format(pr1))
pr2 = data.loc[(data.children_cat == 'нет детей')&(data.debt == 1), 'debt'].sum()/data[data.children_cat == 'нет детей'].shape[0]
print('% проблемных клиентов среди тех, у нет детей', '{:.1%}'.format(pr2))
print()
for x in [1, 2, 3, 4, 5]:
    c = data.loc[(data['children'] == x)&(data.debt == 1), 'debt'].sum()/data[data['children'] == x].shape[0]
    print('% проблемных клиентов среди тех, у кого кол-во детей', x, ' -  ' '{:.1%}'.format(c))
 
#группа клиентов с 5 детьми совсем малочисленна - всего 9 человек 

% проблемных клиентов среди тех, у кого есть дети 9.2%
% проблемных клиентов среди тех, у нет детей 7.5%

% проблемных клиентов среди тех, у кого кол-во детей 1  -  9.2%
% проблемных клиентов среди тех, у кого кол-во детей 2  -  9.5%
% проблемных клиентов среди тех, у кого кол-во детей 3  -  8.2%
% проблемных клиентов среди тех, у кого кол-во детей 4  -  9.8%
% проблемных клиентов среди тех, у кого кол-во детей 5  -  0.0%


In [29]:
# По абсолютным цифрам также видна зависимость - должники чаще встречаются среди клиентов с детьми.
# Данные по многодетным семьям стоит рассматривать справочно, так как эта группа очень малочисленна (менее 3% от общего числа).

tab = pd.DataFrame({'client_q-ty':data.children.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.children).debt.sum()}, 
                    index= data.children.value_counts().index)
tab['%_issue_cl'] =  tab['issue_client_q-ty'] / tab['client_q-ty'] 
tab.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
0,14091,1063,7.5%
1,4855,445,9.2%
2,2128,202,9.5%
3,330,27,8.2%
4,41,4,9.8%
5,9,0,0.0%


In [30]:
tab_ = pd.DataFrame({'client_q-ty':data.children_cat.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.children_cat).debt.sum()}, 
                    index= data.children_cat.value_counts().index)
tab_['%_issue_cl'] =  tab_['issue_client_q-ty'] / tab_['client_q-ty'] 
tab_.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
нет детей,14091,1063,7.5%
есть дети,7363,678,9.2%


In [31]:
print('Процент проблемных клиентов (имели задолженность по кредиту) по всему массиву', 
                                                        '{0:.1%}'.format(data.debt.value_counts()[1]/data.shape[0]))

Процент проблемных клиентов (имели задолженность по кредиту) по всему массиву 8.1%


Среди клиентов с детьми число проблемных больше. Наличие детей увеличивает риск невозврата кредита в срок. Вероятно ответственность за детей снижает значимость хорошей кредитной истории.

**- Семейное положение**

In [32]:
for x in data['family_status'].unique():
    c = data.loc[(data['family_status'] == x)&(data.debt == 1), 'debt'].sum()/data[data['family_status'] == x].shape[0]
    print('% проблемных клиентов среди тех, кто', x, '{:.1%}'.format(c))

% проблемных клиентов среди тех, кто женат / замужем 7.5%
% проблемных клиентов среди тех, кто гражданский брак 9.3%
% проблемных клиентов среди тех, кто вдовец / вдова 6.6%
% проблемных клиентов среди тех, кто в разводе 7.1%
% проблемных клиентов среди тех, кто не женат / не замужем 9.8%


In [33]:
tab1 = pd.DataFrame({'client_q-ty':data.family_status.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.family_status).debt.sum()}, 
                    index= data.family_status.value_counts().index)
tab1['%_issue_cl'] =  tab1['issue_client_q-ty'] / tab1['client_q-ty'] 
tab1.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
женат / замужем,12339,931,7.5%
гражданский брак,4151,388,9.3%
не женат / не замужем,2810,274,9.8%
в разводе,1195,85,7.1%
вдовец / вдова,959,63,6.6%


In [34]:
# Категория про опыт семейной жизни интересна, но из цифр выше видно,
# что на дисплинированность клиентов влияет именно опыт зарегистрированных отношений.

for x in data['family_cat'].unique():
    c = data.loc[(data['family_cat'] == x)&(data.debt == 1), 'debt'].sum()/data[data['family_cat'] == x].shape[0]
    print('% проблемных клиентов в категории', x, '{:.1%}'.format(c))

% проблемных клиентов в категории был опыт семейной жизни 7.9%
% проблемных клиентов в категории не было опыта семейной жизни 9.8%


In [35]:
tab2 = pd.DataFrame({'client_q-ty':data.family_cat.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.family_cat).debt.sum()}, 
                    index= data.family_cat.value_counts().index)
tab2['%_issue_cl'] =  tab2['issue_client_q-ty'] / tab2['client_q-ty'] 
tab2.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
был опыт семейной жизни,18644,1467,7.9%
не было опыта семейной жизни,2810,274,9.8%


Отсутствие опыта семейной жизни в официальном браке увеливает риск невозврата кредита в срок. А зарегистрированные отношения повышают дисциплинированность. 

**- Уровень дохода**

In [36]:
for x in data['income_group'].unique():
    c = data.loc[(data['income_group'] == x)&(data.debt == 1), 'debt'].sum()/data[data['income_group'] == x].shape[0]
    print('% проблемных клиентов с доходом', x, ' -', '{:.1%}'.format(c))

% проблемных клиентов с доходом (4) 195,8+  - 7.1%
% проблемных клиентов с доходом (2) 107,6-142,6  - 8.8%
% проблемных клиентов с доходом (3) 142,6-195,8  - 8.5%
% проблемных клиентов с доходом (1) до 107,6  - 8.0%


In [37]:
tab3 = pd.DataFrame({'client_q-ty':data.income_group.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.income_group).debt.sum()}, 
                    index= data.income_group.value_counts().index)
tab3['%_issue_cl'] =  tab3['issue_client_q-ty'] / tab3['client_q-ty'] 
tab3.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
"(2) 107,6-142,6",5481,483,8.8%
"(4) 195,8+",5365,383,7.1%
"(1) до 107,6",5363,427,8.0%
"(3) 142,6-195,8",5245,448,8.5%


Клиенты с невысоким доходом и высоким более дисциплинированы, чем те, доход которых ближе к среднему. Клиенты, у которых денег немного, по всей видимости, тщательнее планируют свои расходы. Обеспеченые клиенты более свободны в тратах и вероятно также лучше умеют управлять финансами. 

**- Цель кредита**

In [38]:
for x in list(data['purpose_category'].unique()):
    c = data.loc[(data['purpose_category'] == x)&(data.debt == 1), 'debt'].sum()/data[data['purpose_category'] == x].shape[0]
    print('% проблемных клиентов среди тех, цель которых', x, '{:.1%}'.format(c))

% проблемных клиентов среди тех, цель которых недвижимость 7.3%
% проблемных клиентов среди тех, цель которых автомобиль 9.4%
% проблемных клиентов среди тех, цель которых образование 9.2%
% проблемных клиентов среди тех, цель которых свадьба 8.0%
% проблемных клиентов среди тех, цель которых ремонт 5.8%


In [39]:
tab4 = pd.DataFrame({'client_q-ty':data.purpose_category.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.purpose_category).debt.sum()}, 
                    index= data.purpose_category.value_counts().index)
tab4['%_issue_cl'] =  tab4['issue_client_q-ty'] / tab4['client_q-ty'] 
tab4.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
недвижимость,10204,747,7.3%
автомобиль,4306,403,9.4%
образование,4013,370,9.2%
свадьба,2324,186,8.0%
ремонт,607,35,5.8%


In [40]:
# Люди за 40 вероятно тщательнее планируют финансы. Эрелый человек видит меньше перспектив в силу опыта, состояния
# здоровья, отношения на рынке труда к возрасту, ответственности уже не только за детей, но и родителей и пр.
tab5 = pd.DataFrame({'client_q-ty':data.age_group.value_counts(), 
                    'issue_client_q-ty':data.groupby(data.age_group).debt.sum()}, 
                    index= data.age_group.value_counts().index)
tab5['%_issue_cl'] =  tab5['issue_client_q-ty'] / tab5['client_q-ty'] 
tab5.style.format({'%_issue_cl': '{:,.1%}'.format})

,client_q-ty,issue_client_q-ty,%_issue_cl
43-53,5448,399,7.3%
до 33,5366,587,10.9%
34-42,5365,460,8.6%
54+,5275,295,5.6%


Задолженность по кредиту чаще встречается среди клиентов, цель которых - покупка автомобиля. Для России до сих пор характерно показное потребление. Дорогой автомобиль - один из ярких атрибутов статуса. Покупается он вероятно часто с завышенной планкой, но не всегда отрабатывает свою роль. Ситуацию с образованием можно объяснить его переоценной ролью и завышенными ожиданиями (люди вероятно хорошо покупаются на рекламу, но оказывается, что еще нужно учиться, а еще нужен опыт). 

## Заключение

Такие значения признаков как
 - наличие детей
 - отсутствие опыта семейной жизни в официальном браке 
 - доход в диапозоне (107,6 тр, 195,8 тр)
 - цель кредита - покупка автомобиля, образование 
 - возраст до 43 лет  
 
увеличивают риск невозврата кредита в срок (процент проблемных клиентов в этих группах выше среднего).

Исходный массив данных содержит пропуски, неверные значения, одинаковые строки. Если ранее не оценивалось влияние точности формирования массива на качество скоринга, то это стоит сделать. 